# Deep Learning Using LSTM

### Text Cleaning ,Encoding and Padding

In [ ]:
!pip install tensorflow
!pip install scikit-learn
!pip install pandas
!pip install numpy
!pip install seaborn
!pip install matplotlib
!pip install wordcloud
!pip install nltk

In [2]:
import tensorflow as tf
from tensorflow import keras

## Import Libraries

In [3]:
#Deep Learning Packages
from keras.models import Sequential
from keras.layers import Embedding,LSTM, Dense,Dropout
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
#Machine Learning Packages
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import pickle
import nltk
import re
from nltk.stem import PorterStemmer
import seaborn as sns

## Load Data

In [20]:
train_data = pd.read_csv("Emotion.txt",sep=';',header=None,names=['Comments','Emotion'],encoding='utf-8')
train_data.head(5)

,Comments,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [22]:
train_data['length']=[len(x) for x in train_data['Comments']]

In [23]:
max(len(w) for w in train_data['Comments'])

300

In [7]:
stopwords = set(nltk.corpus.stopwords.words('english'))

In [39]:
def clean_text(df,column,vocab_size,max_length):
    stemmer = PorterStemmer()

    corpus = []
    for text in df[column]:
        text = re.sub("[^a-zA-Z]"," ",text)
        text = text.lower()
        text = text.split()
        text = [stemmer.stem(w) for w in text if w not in stopwords] 
        text = " ".join(text)
        corpus.append(text)
    one_hot_word =[one_hot(input_text=word,n=vocab_size) for word in corpus]
    pad = pad_sequences(sequences=one_hot_word,maxlen = max_length,padding='pre')
    return pad
        
x_train = clean_text(train_data,'Comments',vocab_size=11000,max_length=300)

In [40]:
x_train

array([[    0,     0,     0, ...,  9970,  8200,  9704],
       [    0,     0,     0, ...,  8901,  1318,  5566],
       [    0,     0,     0, ...,  8200,  4609,  3705],
       ...,
       [    0,     0,     0, ...,  2819, 10512,  4762],
       [    0,     0,     0, ...,  8332,  1820,  1075],
       [    0,     0,     0, ...,  8200,  9523,  5046]])

In [41]:
train_data['Emotion']

0        4
1        4
2        0
3        3
4        0
        ..
15995    4
15996    4
15997    2
15998    0
15999    4
Name: Emotion, Length: 16000, dtype: int64

In [42]:
le = LabelEncoder()
train_data['Emotion']=le.fit_transform(train_data['Emotion'])

In [43]:
train_data

,Comments,Emotion,length
0,i didnt feel humiliated,4,23
1,i can go from feeling so hopeless to so damned...,4,108
2,im grabbing a minute to post i feel greedy wrong,0,48
3,i am ever feeling nostalgic about the fireplac...,3,92
4,i am feeling grouchy,0,20
...,...,...,...
15995,i just had a very brief time in the beanbag an...,4,101
15996,i am now turning and i feel pathetic that i am...,4,102
15997,i feel strong and good overall,2,30
15998,i feel like this was such a rude comment and i...,0,59


In [44]:
y_train = to_categorical(train_data['Emotion'])

### Model Building and Training

In [46]:
model = Sequential()
model.add(Embedding(input_dim=11000,output_dim=150,input_length=300))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(64,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(6,activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,epochs =10,batch_size=64,verbose=1)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 165s 647ms/step - accuracy: 0.3053 - loss: 1.6499
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 126s 506ms/step - accuracy: 0.6949 - loss: 0.9088
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 121s 482ms/step - accuracy: 0.8841 - loss: 0.3567
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 129s 514ms/step - accuracy: 0.9227 - loss: 0.2299
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 123s 490ms/step - accuracy: 0.9434 - loss: 0.1701
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 127s 509ms/step - accuracy: 0.9594 - loss: 0.1195
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 137s 549ms/step - accuracy: 0.9595 - loss: 0.1168
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 127s 506ms/step - accuracy: 0.9690 - loss: 0.0891
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 135s 539ms/step - accuracy: 0.9751 - loss: 0.0724
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 119s 477ms/step - accuracy: 0.9741 - loss: 0.0689


### Predictive Syetem

In [51]:
def pred(text):
    stemmer = PorterStemmer()
    corpus = []
    text = re.sub("[^a-zA-Z]"," ",text)
    text = text.lower()
    text = text.split()
    text = [stemmer.stem(w) for w in text if w not in stopwords] 
    text = " ".join(text)
    corpus.append(text)
    one_hot_word =[one_hot(input_text=word,n=11000) for word in corpus]
    pad = pad_sequences(sequences=one_hot_word,maxlen = 300,padding='pre')
    return pad

sentences =[
    'I hate him',
    'I feel strong and good Overall',
    'She is really sweet and caring'
]
 
for sent in sentences:
    clean_sent = pred(sent)
    emotion = le.inverse_transform(np.argmax(model.predict(clean_sent),axis=-1))[0]
    label = np.max(model.predict(clean_sent))
    print(emotion)
    print(label)
    print("=====================")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
0
0.96700555
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
2
0.9990846
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
3
0.9179035


### Save the model and the files

In [52]:
model.save('model.h5')